In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras as K
print(tf.__version__)
print(tf.config.list_physical_devices())
!pwd

2.4.1
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
/content


In [ ]:
def recall_m(y_true, y_pred):
    K = tf.keras.backend
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    K = tf.keras.backend
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    K = tf.keras.backend
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
%cd /
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

!ln -s /content/gdrive/My\ Drive/cv_final_project /gd
!ls /gd
!mkdir -p /w
%cd /w

/
Mounted at /content/gdrive
'Copy of 2+.zip'
'Copy of Dataset-v2.0.zip'
 correct_data.7z
 correct_data_aug.7z
 data_aug.7z
 Dataset.7z
 modelv5.h5
 v_1_resnet50_ft40.ipynb
 v_2_resnet50.ipynb
 v_3_xception_sgd.ipynb
 v_4_xception_adam.ipynb
 v_5_xception_adam_lighting_aug.ipynb
 v_6_xception_adam_lighting_aug_separate_data_dirs.ipynb
 weightsv6.h5
/w


In [ ]:
!cp /gd/correct_data_aug.7z /w
!7z x /w/correct_data_aug.7z -o/w/Dataset


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs AMD EPYC 7B12 (830F10),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /w/             1 file, 139310150 bytes (133 MiB)

Extracting archive: /w/correct_data_aug.7z
--
Path = /w/correct_data_aug.7z
Type = 7z
Physical Size = 139310150
Headers Size = 64668
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      2% 13         4% 200 - correct_data_aug/test/0/1856.jpg                                             6% 389 - correct_data_aug/train/0/0133.jpg                                              8% 390 - correct_data_aug/train/0/0134.jpg                                            

In [ ]:
!ls Dataset/correct_data_aug

test  train  validation


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input

img_height, img_width = 224, 224
batch_size = 64
nb_epochs = 60
nb_classes = 3

train_data_dir = '/w/Dataset/correct_data_aug/train'
validation_data_dir = '/w/Dataset/correct_data_aug/validation'
test_data_dir = '/w/Dataset/correct_data_aug/test'

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.25,
    width_shift_range=0.25,
    height_shift_range=0.25,
    zoom_range=0.2,
    rotation_range=40
)

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    )

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    )

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,
    class_mode='categorical',
    )

Found 5627 images belonging to 3 classes.
Found 271 images belonging to 3 classes.
Found 276 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.applications import ResNet50, ResNet50V2, Xception
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, AveragePooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, Adadelta, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

def build_model_fine_tune():
    baseModel = Xception(
        include_top=False,
        weights="imagenet",
        input_tensor=Input(shape=(img_height, img_width, 3)),
        pooling='avg',
    )
    
    # for layer in baseModel.layers:
    #     layer.trainable = False
    # for layer in baseModel.layers[-40:]:
    #     layer.trainable = True

    headModel = baseModel.output
    headModel = Dropout(0.2)(headModel)
    headModel = Dense(1024, activation="relu")(headModel)
    headModel = Dropout(0.2)(headModel)
    headModel = Dense(1024, activation="relu")(headModel)
    headModel = Dropout(0.2)(headModel)
    headModel = Dense(512, activation="relu")(headModel)
    headModel = Dropout(0.2)(headModel)
    headModel = Dense(512, activation="relu")(headModel)
    headModel = Dropout(0.2)(headModel)
    headModel = Dense(nb_classes, activation="softmax")(headModel)
    
    model = Model(inputs=baseModel.input, outputs=headModel)

    return model


learning_rate = 0.0001
weights = os.path.join('/gd', 'weightsv6.h5')
callbacks = [ EarlyStopping(monitor='val_f1_m', patience=20, verbose=1, mode='max'), 
              ModelCheckpoint(weights, monitor='val_f1_m', save_best_only=True, verbose=1, mode='max'),
              ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20, verbose=1, mode='min', min_delta=0.0001, cooldown=0, min_lr=0)]
loss = CategoricalCrossentropy()
# optimizer = SGD(lr=1e-2, momentum=0.9, decay=1e-2/40)
optimizer = Adam(lr=learning_rate)
model = build_model_fine_tune()
model.compile(
    loss=loss, 
    optimizer='adam', 
    metrics=['accuracy', f1_m, precision_m, recall_m],
)
 
history = model.fit(
    train_generator, 
    validation_data=validation_generator, 
    epochs=nb_epochs, 
    batch_size=batch_size, 
    callbacks=callbacks,
)

Epoch 1/60
88/88 [==============================] - 117s 1s/step - loss: 0.6432 - accuracy: 0.7211 - f1_m: 0.6903 - precision_m: 0.8225 - recall_m: 0.6073 - val_loss: 6.8004 - val_accuracy: 0.7601 - val_f1_m: 0.7522 - val_precision_m: 0.7632 - val_recall_m: 0.7427

Epoch 00001: val_f1_m improved from -inf to 0.75223, saving model to /gd/weightsv6.h5
Epoch 2/60
88/88 [==============================] - 110s 1s/step - loss: 0.2112 - accuracy: 0.9204 - f1_m: 0.9201 - precision_m: 0.9377 - recall_m: 0.9036 - val_loss: 0.4127 - val_accuracy: 0.8930 - val_f1_m: 0.9111 - val_precision_m: 0.9195 - val_recall_m: 0.9031

Epoch 00002: val_f1_m improved from 0.75223 to 0.91110, saving model to /gd/weightsv6.h5
Epoch 3/60
88/88 [==============================] - 110s 1s/step - loss: 0.1279 - accuracy: 0.9634 - f1_m: 0.9642 - precision_m: 0.9689 - recall_m: 0.9596 - val_loss: 1.4199 - val_accuracy: 0.8118 - val_f1_m: 0.8400 - val_precision_m: 0.8425 - val_recall_m: 0.8375

Epoch 00003: val_f1_m did n

In [ ]:
best_model = tf.keras.models.load_model('/gd/weightsv6.h5', custom_objects={'f1_m':f1_m, 'precision_m':precision_m, 'recall_m':recall_m})
best_model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy', f1_m, precision_m, recall_m],
)

In [ ]:
best_model.evaluate(test_generator, batch_size=batch_size)

5/5 [==============================] - 51s 10s/step - loss: 0.2380 - accuracy: 0.9638 - f1_m: 0.9688 - precision_m: 0.9688 - recall_m: 0.9688


[0.23804134130477905, 0.9637681245803833, 0.96875, 0.96875, 0.96875]

In [ ]:
best_model.save("/w/modelv5.h5")

In [ ]:
download("/gd/modelv5.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = best_model.predict(test_generator)
# print(Y_pred)
y_pred = np.argmax(Y_pred, axis=1)
print(y_pred)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['0', '1', '2']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 2 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 0 1 1 1 0 2 2 0 1 1 1 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
Confusion Matrix
[[186   2   1]
 [  3  22   4]
 [  0   0  58]]
Classification Report
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       189
           1       0.92      0.76      0.83        29
           2       0.92      1.00      0.96        58

    accuracy                           0.96       276
   macro avg       0.94      0.91      0.92       276
weighted avg       0.96     